In [49]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [50]:
import matplotlib.pyplot as plt
from jax import random
from PFNExperiments.LatentFactorModels.GenerativeModels.Numpyro_Versions.LDA_NumPyro import make_lda_program, make_lda_program_batched

In [51]:
from PFNExperiments.LatentFactorModels.GenerativeModels.LatenFactorModel_abstract import return_only_x
from PFNExperiments.LatentFactorModels.ComparisonModels.Hamiltionian_MC import Hamiltionian_MC

from PFNExperiments.LatentFactorModels.ComparisonModels.MakeDefaultListComparison import make_default_list_comparison

In [52]:
import numpyro.distributions as dist
import numpyro

In [53]:
pprogram = make_lda_program(
    n_docs = 100,
    n_words = 500,
    n_topics = 3,
    doc_len_max = 20
    )

In [54]:
r = pprogram(
)

x = r["x"]

In [55]:
r["x"]

Array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [56]:
for key, value in r.items():
    print(key, value.shape)

theta (100, 3)
phi (3, 500)
doc_len (100,)
x (100, 500)


In [57]:
from PFNExperiments.LatentFactorModels.ComparisonModels.Hamiltionian_MC_Numpyro import Hamiltionian_MC

In [58]:
hmc = Hamiltionian_MC(pprogram=pprogram, n_warmup = 500, n_samples=500)

In [59]:
samples = hmc.sample_posterior(
    x = x,
)

C:\Users\arik_\Documents\Dokumente\Job_Clausthal\PFNs\Repository\PFNExperiments\LatentFactorModels\ComparisonModels\Hamiltionian_MC_Numpyro.py:65: FutureWarning: Some algorithms will automatically enumerate the discrete latent site doc_len of your model. In the future, enumerated sites need to be marked with `infer={'enumerate': 'parallel'}`.
  self.mcmc.run(rng_key=self.sample_key, x=x)


KeyboardInterrupt: 

In [157]:
samples["theta"].shape

torch.Size([500, 100, 10])

In [172]:
# try vi

from numpyro.infer import SVI, TraceGraph_ELBO, TraceEnum_ELBO

from numpyro.infer.autoguide import AutoBNAFNormal
guide = AutoBNAFNormal(pprogram)

optim = numpyro.optim.Adam(step_size=0.01)

svi = SVI(
    model=pprogram,
    guide=guide,
    optim=optim,
    loss=TraceGraph_ELBO()
)

In [173]:
svi_result = svi.run(rng_key=random.PRNGKey(0), x=x, num_steps=1000)

C:\Users\arik_\AppData\Local\Temp\ipykernel_21420\1965272369.py:1: FutureWarning: Some algorithms will automatically enumerate the discrete latent site doc_len of your model. In the future, enumerated sites need to be marked with `infer={'enumerate': 'parallel'}`.
  svi_result = svi.run(rng_key=random.PRNGKey(0), x=x, num_steps=1000)


ValueError: Continuous inference cannot handle discrete sample site 'doc_len'.

In [45]:
pprogram_batched = make_lda_program_batched(
    n_docs = 20,
    n_words = 100,
    n_topics = 3,
    batch_size=32,
    doc_len_max = 20
)

In [46]:
rb = pprogram_batched()

In [47]:
x_res = rb["x"]

In [48]:
x_res.sum()

tensor(6333.)

In [31]:
import torch

In [32]:
x_res_zeros = torch.zeros_like(x_res)

In [33]:
rb2 = pprogram_batched()

6447.0


In [34]:
rb2["x"].sum()

tensor(6447.)